In [9]:
from dotenv import load_dotenv,find_dotenv
import os
env_path = find_dotenv(filename='password.env',usecwd=True)
load_dotenv(env_path)

True

In [10]:
key = os.getenv('OPENAI_API_KEY')


In [ ]:
# Load a pdf document
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('rag-paper.pdf')
data = loader.load()
print(data[0].page_content[:500])

Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
Patrick Lewis†‡, Ethan Perez⋆,
Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,
Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†
†Facebook AI Research; ‡University College London; ⋆New York University;
plewis@fb.com
Abstract
Large pre-trained language models have been shown to store factual knowledge
in their parameters, and achieve state-of-the-art results when


In [24]:
# splitting the document into chunks
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=10,
                                      separator = "\n\n")
chunks = text_splitter.split_documents(data)
print(f'Number of chunks: {len(chunks)}')
[len(chunk.page_content) for chunk in chunks]

Number of chunks: 19


[2900,
 4573,
 3658,
 4206,
 4556,
 4110,
 4331,
 3186,
 4124,
 3778,
 3767,
 3990,
 4010,
 3984,
 4006,
 1138,
 2558,
 4243,
 1962]

In [37]:
#Splitting using RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
                                               chunk_overlap=10,
                                               separators=["\n\n", "\n", " ", ""])
chunks = text_splitter.split_documents(data)
print(f'Number of chunks: {len(chunks)}')
[len(chunk.page_content) for chunk in chunks]

Number of chunks: 876


[95,
 89,
 78,
 95,
 85,
 81,
 82,
 80,
 84,
 80,
 80,
 77,
 78,
 75,
 73,
 78,
 80,
 78,
 81,
 76,
 79,
 77,
 77,
 82,
 32,
 99,
 96,
 95,
 96,
 12,
 95,
 98,
 92,
 90,
 97,
 40,
 98,
 83,
 98,
 96,
 98,
 98,
 86,
 96,
 96,
 92,
 95,
 98,
 96,
 94,
 96,
 93,
 94,
 96,
 97,
 95,
 13,
 97,
 92,
 14,
 97,
 99,
 85,
 19,
 95,
 93,
 93,
 96,
 36,
 96,
 96,
 95,
 92,
 94,
 11,
 98,
 89,
 93,
 96,
 96,
 4,
 96,
 9,
 92,
 93,
 96,
 87,
 19,
 93,
 93,
 94,
 92,
 12,
 33,
 92,
 18,
 95,
 97,
 97,
 98,
 80,
 86,
 96,
 13,
 96,
 97,
 98,
 68,
 87,
 98,
 91,
 92,
 18,
 93,
 9,
 66,
 96,
 12,
 92,
 18,
 92,
 57,
 95,
 96,
 98,
 96,
 99,
 96,
 84,
 19,
 94,
 98,
 92,
 92,
 15,
 97,
 92,
 12,
 94,
 98,
 1,
 95,
 97,
 97,
 92,
 89,
 93,
 87,
 54,
 68,
 44,
 92,
 97,
 10,
 98,
 97,
 93,
 94,
 6,
 95,
 97,
 91,
 96,
 9,
 85,
 96,
 94,
 99,
 92,
 95,
 97,
 92,
 91,
 93,
 99,
 94,
 99,
 89,
 99,
 97,
 92,
 88,
 96,
 84,
 34,
 94,
 98,
 89,
 95,
 99,
 88,
 96,
 96,
 88,
 70,
 32,
 94,
 94,
 96,
 95,
 12,
 

In [44]:
# Creating a vector database and storing documents in it
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts  import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
embedding_model = OpenAIEmbeddings(openai_api_key=key,
                                   model="text-embedding-3-small")
vectordb = Chroma.from_documents(documents=chunks,
                                    embedding=embedding_model)
#Define retreiver
retreiver = vectordb.as_retriever(search_type="similarity",
                                  search_kwargs={"k":2})
#define llm
llm = ChatOpenAI(openai_api_key=key,
                 model_name = "gpt-4o-mini",
                 temperature = 0)

#Create a prompt template
prompt = ChatPromptTemplate.from_template("Use the only the following context to answer the question:{context}\n Question:{question}\n Answer:")
rag_chain = ({"context":retreiver,
             "question":RunnablePassthrough()} | prompt | llm | StrOutputParser())

#test the RAG chain
result = rag_chain.invoke("Explain the concept of retrieval-augmented generation in 3 bullet points")

In [45]:
print(result)

- **Definition**: Retrieval-augmented generation (RAG) is a fine-tuning approach that combines retrieval mechanisms with generative models to enhance the quality and relevance of generated content.

- **Mechanism**: RAG retrieves relevant information from a knowledge base or dataset during the generation process, allowing the model to incorporate external knowledge and context into its responses.

- **Applications**: This approach is particularly useful in tasks such as question answering, summarization, and dialogue systems, where accurate and contextually appropriate information is crucial for generating meaningful outputs.


In [58]:
#Step1: Load a python file
from langchain_community.document_loaders import PythonLoader
loader = PythonLoader('chatbot.py')
data = loader.load()

#Step 2 split the documents
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language 
splitter = RecursiveCharacterTextSplitter.from_language(
    language = Language.PYTHON,
    chunk_size = 100,
    chunk_overlap = 10
)
chunks = splitter.split_documents(data)
for i,chunk in enumerate(chunks[:2]):
    print(f'--- Chunk {i} ---')
    print(chunk.page_content)

#Step 3: Create a vector database and store the documents in it
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
embediing_model = OpenAIEmbeddings(openai_api_key=key,
                                  model="text-embedding-3-small")
vectordb = Chroma.from_documents(documents=chunks,
                                   embedding=embediing_model)   

#step4 Define a retreiver and an llm
from langchain_openai import ChatOpenAI
retreiver = vectordb.as_retriever(search_type="similarity",
                                  search_kwargs={"k":2})
llm = ChatOpenAI(openai_api_key=key,
                 model_name = "gpt-4o-mini",
                    temperature = 0)
#Step 5: Create a prompt template and RAG chain
from langchain_core.prompts  import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Use only the following context to answer the question:" \
"context:{context}\n" \
" Question:{question}\n" \
    " Answer:" )
rag_chain = ({"context":retreiver,
             "question":RunnablePassthrough()} | prompt | llm | StrOutputParser())

#Step 6: Test the RAG chain
result = rag_chain.invoke("Explain the functions in the chatbot.py file in brief")
with open('rag_output.txt','w') as f:
    f.write(result)

--- Chunk 0 ---
from abc import ABC, abstractmethod
--- Chunk 1 ---
class LLM(ABC):
  @abstractmethod
  def complete_sentence(self, prompt):
    pass


In [57]:
print(result)

The `ChatBot` class in the provided code contains a method called `_get_llm`, which is responsible for returning a language model (LLM) based on the specified provider. In this case, if the provider is "OpenAI", the method returns an instance of the `OpenAI` class. This suggests that the `ChatBot` class is designed to interact with different language model providers, allowing for flexibility in choosing which model to use for generating responses.
